In [6]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds

#MNIST data set
datos, metadatos = tfds.load('emnist', as_supervised=True, with_info=True)

datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #0-255 to 0-1
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#creating the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,
                kernel_size=(5, 5),
                strides=(2, 2),
                input_shape=(28, 28, 1),
                activation='relu'))
model.add(tf.keras.layers.Conv2D(64,
                kernel_size=(3, 3),
                activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(65, activation=tf.nn.softmax))

#Compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 32

#shuffling
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#training
historial = model.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

Epoch 1/5
15065/21811 [===================>..........] - ETA: 2:23 - loss: 0.5916 - accuracy: 0.8049

KeyboardInterrupt: ignored

In [ ]:
#exporting the model to h5
modelo.save('numeros.h5')

In [ ]:
!ls

'~'   numeros.h5   sample_data


In [ ]:
!pip install tensorflow js

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 53.6 MB/s 
  Created wheel for js: filename=js-1.0-py3-none-any.whl size=2907 sha256=6ca839466808093b2d45c35e2cd0f05bf0b22bb25e5faf11a0864cb9caa3c6c9
  Stored in directory: /root/.cache/pip/wheels/bf/92/56/6597c8f32980ee64af7e864796e8d7423eb67eafff5415a4ae
  Created wheel for shutilwhich: filename=shutilwhich-1.1.0-py3-none-any.whl size=2781 sha256=a9ab1b63d62cd78b9779aa46cd4bdf96a699112215292231be24103672cb3c40
  Stored in directory: /root/.cache/pip/wheels/4c/c0/ce/5a47f35186acbe73e38811df30daaf798c81a7599bb2d5995a
Successfully built js shutilwhich


In [ ]:
#folder with output files
!mkdir carpeta_salida

In [ ]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

/bin/bash: tensorflowjs_converter: command not found


In [ ]:
!ls carpeta_salida

In [ ]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html